# ゼロから作る Deep Learning 4 強化学習編 勉強ノート 第9章 〜方策勾配法〜

これまで学んだ Q 学習や SARSA やモンテカルロ法などは、大別すれば<font color="red">**価値ベースの手法**</font>(Value-based Method)に分類される。  
これらの価値ベースの手法では価値関数をモデル化し、価値関数を学習する。  
そして価値関数を経由して方策を得る。  
これらは<font color="red">**一般化方策反復**</font>というアイデアに基づいて最適方策を見つけることが多く行われる。

一方で、<font color="red">**方策ベースの手法**</font>(Policy-based Method)は価値関数を経由せずに方策を直接表す。  
方策をニューラルネットワークなどでモデル化し、勾配を使って方策を最適化する手法は<font color="red">**方策勾配法**</font>(Policy Gradient Method)と呼ばれる。  
ここでは、最も単純な方策勾配法から徐々に改善する流れで、様々な手法を扱う。

## 最も単純な方策勾配法

方策勾配法は、勾配を使った方策を更新する手法の総称である。  
ここでは方策勾配法の中で最も単純なものを導出する。

### 方策勾配法の導出

確率的な方策は $\pi(a|s)$ と表される。  
$\pi(a|s)$ は状態 $s$ において $a$ という行動を取る確率を表す。  
方策をニューラルネットワークでモデル化するにあたって、全ての重みのパラメータを $\theta$ で集約して表すことにする。  
そして、ニューラルネットワークによる方策を $\pi_{\theta}(a|s)$ と表すことにする。  
まず、次のような「状態、行動、報酬」からなる時系列データが得られたエピソードタスクを考える。

$$\tau = (S_0, A_0, R_0, S_1, A_1, \cdots, S_{T+1})$$

$\tau$ は<font color="red">**軌道**</font>(Trajectory)とも呼ばれる。  
このとき、収益は割引率 $\gamma$ を使って

$$G(\tau) = R_0 + \gamma R_1 + \gamma^2 R_2 + \cdots + \gamma^T R_T$$

と設定できる。  
ここでは、収益が $\tau$ から計算できることを明示するために $G(\tau)$ と表記している。  
このとき、目的関数 $J(\theta)$ は

$$J(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}}[G(\tau)]$$

と表される。  
収益 $G(\tau)$ は確率的に変動するので、その期待値が目的関数となる。  
目的関数が決まれば、次はその勾配を求める。  
ここではパラメータ $\theta$ に関する勾配を $\nabla_\theta$ で表すことにする。

\begin{eqnarray*}
\nabla_\theta J(\theta) &=& \nabla_\theta \mathbb{E}_{\tau \sim \pi_{\theta}}[G(\tau)] \\
&=& \mathbb{E}_{\tau \sim \pi_{\theta}} \left[ \sum_{t=0}^T G(\tau) \nabla_\theta \log \pi_{\theta}(A_t|S_t) \right]
\end{eqnarray*}

そして、ニューラルネットワークのパラメータの更新は、

$$\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)$$

と勾配方向に学習率 $\alpha$ だけ更新する。

### 方策勾配法のアルゴリズム

上記で求めた期待値 $\nabla_\theta J(\theta)$ はモンテカルロ法によって求められる。  
方策 $\pi_{\theta}$ のエージェントに実際に行動させ、軌道 $\tau$ を $n$ 個得たとする。  
その場合、各 $\tau$ において $\sum_{t=0}^T G(\tau) \nabla_\theta \log \pi_{\theta}(A_t|S_t)$ を計算して、その平均を求めることで $\nabla_\theta J(\theta)$ を近似できる。

$$
\text{sampling: } \tau^{(i)} \sim \pi_{\theta} \quad (i = 1, 2, \cdots, n) \\
x^{(i)} = \sum_{t=0}^T G(\tau^{(i)}) \nabla_\theta \log \pi_{\theta} \left( A_t^{(i)} | S_t^{(i)} \right) \\
\nabla_\theta J(\theta) \simeq \frac{x^{(1)} + x^{(2)} + \cdots + x^{(n)}}{n}
$$

ここでは $i$ 番目のエピソードで得られた軌道を $\tau^{(i)}$ 、 $i$ 番目のエピソードの時刻 $t$ における行動を $A_t^{(i)}$ 、状態を $S_t^{(i)}$ で表すことにする。  
$n = 1$ の場合を考えると、

$$
\text{sampling: } \tau \sim \pi_{\theta} \\
\nabla_\theta J(\theta) \simeq \sum_{t=0}^T G(\tau) \nabla_\theta \log \pi_{\theta}(A_t|S_t)
$$

である。  
この式は、 $\nabla_\theta \log \pi_{\theta}(A_t|S_t)$ を全ての時刻 $(t = 0 \sim T)$ で求め、各勾配に「重み」として収益 $G(\tau)$ をかけてそれらの和を求める。  
すなわち、

$$G(\tau) \nabla_\theta \log \pi_{\theta}(A_0|S_0) + G(\tau) \nabla_\theta \log \pi_{\theta}(A_1|S_1) + \cdots + G(\tau) \nabla_\theta \log \pi_{\theta}(A_T|S_T)$$

である。  
この計算の「意味」を考える。  
まずは $\log$ の微分により、

$$\nabla_\theta \log \pi_{\theta}(A_t|S_t) = \frac{\nabla_\theta \pi_{\theta}(A_t|S_t)}{\pi_{\theta}(A_t|S_t)}$$

が成り立つ。  
$\nabla_\theta \log \pi_{\theta}(A_t|S_t)$ は、 $\nabla_\theta \pi_{\theta}(A_t|S_t)$ という勾配を $\frac{1}{\pi_{\theta}(A_t|S_t)}$ 倍したものであり、これより $\nabla_\theta \log \pi_{\theta}(A_t|S_t)$ と $\nabla_\theta \pi_{\theta}(A_t|S_t)$ は同じ方向を指すことがわかる。  
$\nabla_\theta \pi_{\theta}(A_t|S_t)$ は、状態 $S_t$ で行動 $A_t$ を取る確率が最も増える方向を指す。  
同じく $\nabla_\theta \log \pi_{\theta}(A_t|S_t)$ も状態 $S_t$ で行動 $A_t$ を取る確率が最も増える方向を指す。  
そこに「重み」 $G(\tau)$ がかけられる。  
これは、上手くいったら上手くいった分だけそれまでに取った行動が強められ、反対に、上手くいかなかった場合、その間に取った行動はその分だけ弱められることを「意味」する。

### 方策勾配法の実装(*コーディング*)

続いて、最も単純な方策勾配法の実装に移る。

In [ ]:
!pip install -U japanize-matplotlib

In [ ]:
# ライブラリのインポート
import gym
import japanize_matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.distributions import Categorical
import torch.nn as nn
import torch.optim as optim

In [ ]:
# ニューラルネットワークの方策を表す Policy クラスの実装
class Policy(nn.Module):
    def __init__(self, action_size):
        super().__init__()
        self.l1 = nn.Linear(4, 128)
        self.l2 = nn.Linear(128, action_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.softmax(x)
        return x


# Agent クラスの実装
class Agent:
    def __init__(self):
        self.gamma = 0.98
        self.lr = 0.0002
        self.action_size = 2

        self.memory = []
        self.pi = Policy(self.action_size)
        self.optimizer = optim.Adam(self.pi.parameters(), lr=self.lr)

    def get_action(self, state):
        state = torch.tensor(state[np.newaxis, :])
        probs = self.pi(state)
        probs = probs[0]
        m = Categorical(probs)
        action = m.sample().item()
        return action, probs[action]

    def add(self, reward, prob):
        data = (reward, prob)
        self.memory.append(data)

    def update(self):
        G, loss = 0, 0
        for reward, prob in reversed(self.memory):
            G = reward + self.gamma * G

        for reward, prob in self.memory:
            loss += - torch.log(prob) * G

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.memory = []

In [ ]:
# 報酬をプロットする関数を定義
def reward_show(history: list) -> None:
    x = [i for i in range(1, len(history) + 1)]

    plt.figure(figsize=(8, 6), tight_layout=True)
    plt.title("報酬の遷移", size=15, color="red")
    plt.grid()
    plt.plot(x, history)

In [ ]:
# 各クラスのインスタンスを生成
env = gym.make("CartPole-v1")
agent = Agent()

# 3000回のエピソードで実行
episodes = 3000
reward_history = []
for episode in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action, prob = agent.get_action(state)
        next_state, reward, done, info = env.step(action)

        agent.add(reward, prob)
        state = next_state
        total_reward += reward

    agent.update()

    reward_history.append(total_reward)
    if episode % 100 == 0:
        print(f"episode :{episode}, total reward : {total_reward}")

# 報酬の遷移画像を描画
reward_show(reward_history)

大きな変動はあるが、エピソードが進むにつれて徐々に良い結果が得られている。

## REINFORCE

前節の方策勾配法では、報酬の総和がエピソードを重ねるごとに向上する一方で、後半の方でも低い値を取ってしまっている。  
REINFORCE はこれを改善した手法であり、ここでは数式ベースでアルゴリズムの導出を行う。  
そして、前のコードを一部修正する形で REINFORCE を実装する。

### REINFORCE アルゴリズム

最も単純な勾配方策法は次の式に基づいて実装される。

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}} \left[ \sum_{t=0}^T G(\tau) \nabla_\theta \log \pi_{\theta}(A_t|S_t) \right]
$$

エージェントの行動の良し悪しは、その行動の<font color="red">**後**</font>に得られた報酬の総和によって評価される。  
逆に、ある行動を起こす<font color="red">**前**</font>に得られた報酬は、その行動の良し悪しとは関係がない。  
行動 $A_t$ に対する重みは $G(\tau)$ であり、これには時刻 $t$ よりも前の報酬も含まれる。  
これを改善するには、重み $G(\tau)$ を次のように変更することが考えられる。

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}} \left[ \sum_{t=0}^T G_t \nabla_\theta \log \pi_{\theta}(A_t|S_t) \right] \\
G_t = R_t + \gamma R_{t+1} + \cdots + \gamma^{T-1}R_T
$$

これにより、行動 $A_t$ の選ばれる確率は、時刻 $t$ より前の報酬を含まない重み $G_t$ によって強められる。

### REINFORCE の実装(*コーディング*)

REINFORCE は分散が小さいので、データのサンプル数が少なくても精度良く近似できる。  
前節の `Agent` クラスの `update` メソッドのみ変更を加える。

In [ ]:
# REINFORCE 用の Agent クラスの実装
class Agent:
    def __init__(self):
        self.gamma = 0.98
        self.lr = 0.0002
        self.action_size = 2

        self.memory = []
        self.pi = Policy(self.action_size)
        self.optimizer = optim.Adam(self.pi.parameters(), lr=self.lr)

    def get_action(self, state):
        state = torch.tensor(state[np.newaxis, :])
        probs = self.pi(state)
        probs = probs[0]
        m = Categorical(probs)
        action = m.sample().item()
        return action, probs[action]

    def add(self, reward, prob):
        data = (reward, prob)
        self.memory.append(data)

    def update(self):
        G, loss = 0, 0
        for reward, prob in reversed(self.memory):
            G = reward + self.gamma * G
            loss += - torch.log(prob) * G

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.memory = []

In [ ]:
# 各クラスのインスタンスを生成
env = gym.make("CartPole-v1")
agent = Agent()

# 3000回のエピソードで実行
episodes = 3000
reward_history = []
for episode in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action, prob = agent.get_action(state)
        next_state, reward, done, info = env.step(action)

        agent.add(reward, prob)
        state = next_state
        total_reward += reward

    agent.update()

    reward_history.append(total_reward)
    if episode % 100 == 0:
        print(f"episode :{episode}, total reward : {total_reward}")

# 報酬の遷移画像を描画
reward_show(reward_history)

前節の勾配方策法と比べて安定した学習が行われており、上限の500に近づいている。

## ベースライン

REINFORCE を改善するために、<font color="red">**ベースライン**</font>(Baseline)という技術を導入する。

### ベースラインのアイデア

A 、 B 、 C の3人がテストを受けて、それぞれ90点、40点、50点を取ったとする。

|  |  |
| :--: | :--: |
| A | 90 |
| B | 40 |
| C | 50 |

これに対して分散を求めると、466.6667と大きな値を取る。  
この分散を減らす方法を考える。

|  | 1回目のテスト | 2回目のテスト | ... | 10回目のテスト |
| :--: | :--: | :--: | :--: | :--: |
| A | 92 | 80 | ... | 74 |
| B | 32 | 51 | ... | 56 |
| C | 45 | 53 | ... | 49 |

このように、これまでのテスト結果があれば過去のテストを平均することで、次のテスト結果は過去の平均点からの「差分」として予測できる。  
上の表の結果をそれぞれ平均すると、 A は82点、 B は46点、 C は49点になったとする。  
これを「予測値」として使い、対象とするテスト結果との差分について考える。

|  | 今回のテスト | 平均 | 差分 |
| :--: | :--: | :--: | :--: |
| A | 90 | 82 | 8 |
| B | 40 | 46 | -6 |
| C | 50 | 49 | 1 |

差分について分散を求めると、32.6667となり、最初と比べると大きく値を減らすことができた。  
この例で示すように、ある結果に対して予測値を引くことで分散を減らすことができる。  
予測値の精度が高ければ高いほど、分散は小さくなる。  
これがベースラインのアイデアである。

### ベースライン付きの方策勾配法

REINFORCE にベースラインを適用すると、次のようになる。

\begin{eqnarray*}
\nabla_\theta J(\theta) &=& \mathbb{E}_{\tau \sim \pi_{\theta}} \left[ \sum_{t=0}^T G_t \nabla_\theta \log \pi_{\theta}(A_t|S_t) \right] \\
&=& \mathbb{E}_{\tau \sim \pi_{\theta}} \left[ \sum_{t=0}^T (G_t - b(S_t)) \nabla_\theta \log \pi_{\theta}(A_t|S_t) \right]
\end{eqnarray*}

$G_t$ の代わりに $G_t - b(S_t)$ を使うことができる。  
ここで、 $b(S_t)$ は入力を $S_t$ とする任意の関数であり、これがベースラインである。  
実践的によく使われるのは価値関数であり、 $b(S_t) = V_{\pi_{\theta}}(S_t)$ となる。  
ベースラインを使って分散を小さくすることができれば、サンプル効率の良い学習が行える。  
なお、ベースラインとして価値関数を使う場合、真の価値関数 $v_{\pi_{\theta}}(S_t)$ を知ることはできず、この場合、価値関数についても学習をする必要がある。

## Actor-Critic

強化学習のアルゴリズムは、大別すると価値ベースの手法と方策ベースの手法に分けられるが、その両方を使用する手法も考えられる。  
Actor-Critic もその一つであり、ベースライン付きの REINFORCE をさらに推し進めた手法である。

### Actor-Critic の導出

ベースライン付きの REINFORCE では、目的関数の勾配は次の式で表された。

$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}} \left[ \sum_{t=0}^T (G_t - b(S_t)) \nabla_\theta \log \pi_{\theta}(A_t|S_t) \right]$$

ここで、 $G_t$ は収益、 $b(S_t)$ はベースラインを表している。  
ベースラインには任意の関数を用いることができるので、ここでは、ニューラルネットワークでモデル化した価値関数を使う。  
それにあたり、次の記号を新たに使用する。

- $w$ : 価値関数を表すニューラルネットワークの全ての重みパラメータ
- $V_w(S_t)$ : 価値関数をモデル化したニューラルネットワーク

この場合の目的関数の勾配は次の式で表される。

$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}} \left[ \sum_{t=0}^T (G_t - V_w(S_t)) \nabla_\theta \log \pi_{\theta}(A_t|S_t) \right]$$

この式には問題があり、それは収益 $G_t$ はゴールに達しないと値が定まらないことである。  
つまり、ゴールに達するまで方策や価値関数の更新ができないということである。  
これを解決するために TD 法が導入された。  
上記の式を TD 法へ切り替えると次のような式が得られる。

$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}} \left[ \sum_{t=0}^T (R_t + \gamma V_w(S_{t+1}) - V_w(S_t)) \nabla_\theta \log \pi_{\theta}(A_t|S_t) \right]$$

この式に基づく手法が Actor-Critic である。

### Actor-Critic の実装(*コーディング*)

Actor-Critic の実装に移る。

In [ ]:
# 価値関数を表す ValueNet クラスの実装
class ValueNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(4, 128)
        self.l2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        return x


# A 用の Agent クラスの実装
class Agent:
    def __init__(self):
        self.gamma = 0.98
        self.lr_pi = 0.0002
        self.lr_v = 0.0005
        self.action_size = 2

        self.pi = Policy(self.action_size)
        self.v = ValueNet()

        self.optimizer_pi = optim.Adam(self.pi.parameters(), lr=self.lr_pi)
        self.optimizer_v = optim.Adam(self.v.parameters(), lr=self.lr_v)

    def get_action(self, state):
        state = torch.tensor(state[np.newaxis, :])
        probs = self.pi(state)
        probs = probs[0]
        m = Categorical(probs)
        action = m.sample().item()
        return action, probs[action]

    def update(self, state, action_prob, reward, next_state, done):
        state = torch.tensor(state[np.newaxis, :])
        next_state = torch.tensor(next_state[np.newaxis, :])

        target = reward + self.gamma * self.v(next_state) * (1 - done)
        target.detach()
        v = self.v(state)
        loss_fn = nn.MSELoss()
        loss_v = loss_fn(v, target)

        delta = target - v
        loss_pi = -torch.log(action_prob) * delta.item()

        self.optimizer_v.zero_grad()
        self.optimizer_pi.zero_grad()
        loss_v.backward()
        loss_pi.backward()
        self.optimizer_v.step()
        self.optimizer_pi.step()

In [ ]:
# 各クラスのインスタンスを生成
env = gym.make("CartPole-v1")
agent = Agent()

# 3000回のエピソードで実行
episodes = 3000
reward_history = []
for episode in range(2000):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action, prob = agent.get_action(state)
        next_state, reward, done, info = env.step(action)

        agent.update(state, prob, reward, next_state, done)

        state = next_state
        total_reward += reward

    reward_history.append(total_reward)
    if episode % 100 == 0:
        print(f"episode :{episode}, total reward : {total_reward}")

# 報酬の遷移画像を描画
reward_show(reward_history)

グラフを見ると、順調に学習が行われていることがわかる。

## 方策ベースの手法の利点

最後に、方策ベースの利点について説明する。

1. ***方策を直接モデル化するので効率的***  
    価値ベースの手法は、価値関数を推定して、それを基に方策を決める。  
    一方、方策ベースの手法は方策を「直接」推定する。  
    問題によっては価値関数は複雑な形状をしながらも、最適方策は単純な場合に考えられる。  
    そのような場合、方策ベースの手法の方がより速く学習できることが期待できる。


2. ***連続的な行動空間でも使える***  
    これまでのカートポールの場合は、右か左かの2つの行動のどちらかを選ぶ。  
    そのような離散行動空間では、行動はいくつかの候補の中から1つを選択する。  
    一方で、 OpenAI Gym の「Pendulum」のような連続的な行動空間も考えられ、価値ベースの手法では適用が難しくなる。  
    対策として「量子化」するなどで離散化する必要があるが、多くの場合、良い方法は試行錯誤して探す必要がある。  
    しかし、方策ベースでは、たとえばニューラルネットワークの出力が正規分布を想定した場合、ニューラルネットワークは正規分布の平均と分散を出力することが考えられる。  
    その平均と分散を基にサンプリングすることで連続値が得られる。


3. ***行動の選択確率がスムーズに変化する***  
    価値ベースの手法では、エージェントの行動は ε-greedy 法によって選ばれることが多い。  
    その場合、基本的には Q 関数の一番大きな行動が選ばれる。  
    このとき、 Q 関数の更新により最大値となる行動が変わると、行動の取り方が急に変わることになる。  
    一方、方策ベースの手法は、ソフトマックス関数によって各行動の確率が決まる。  
    そのため、方策のパラメータを更新していく過程で各行動の確率はスムーズに変わり、方策勾配法の学習は安定しやすくなる。